In [28]:
# Imports several basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import time
from scipy import stats
import datetime as dt
import gc

sns.set_style('whitegrid')
%matplotlib

Using matplotlib backend: MacOSX


In [81]:
# ignore the warnings
import warnings 
warnings.filterwarnings("ignore")

# import the package of heterogenous classifiers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX

import xgboost as xgb
import lightgbm as lgb
import catboost as cab

In [30]:
data = pd.read_csv('/Users/mac/Desktop/UCB/290-Data-X/Project/Dataset/summary.csv')

In [87]:
data

,sku_id,order_time,sales,sales_sum/2w,sales_sum/month,sales_sum/quarter,comment_sum,comment_mean,comment_mean/2w,comment_mean/month,...,area,area_sum/2w,area_sum/month,area_sum/quarter,price,cate,para_1,para_2,para_3,output
0,10033,2016/5/1,0,0,0,0,0,0,0.0,0.0,...,0,0,0,0,1375.3,30,667.43,-1,-1,0.0
1,10033,2016/5/15,0,0,0,0,0,0,0.0,0.0,...,0,0,0,0,1375.3,30,667.43,-1,-1,0.0
2,10033,2016/5/29,0,0,0,0,0,0,0.0,0.0,...,0,0,0,0,1375.3,30,667.43,-1,-1,0.0
3,10033,2016/6/12,0,0,0,0,0,0,0.0,0.0,...,0,0,0,0,1375.3,30,667.43,-1,-1,0.0
4,10033,2016/6/26,0,0,0,0,0,0,0.0,0.0,...,0,0,0,0,1375.3,30,667.43,-1,-1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73071,9981,2017/3/5,0,13,25,35,3,1,1.0,1.0,...,0,12,22,28,316.3,101,174.05,4,4,12.0
73072,9981,2017/3/19,0,4,18,39,0,0,1.0,1.0,...,0,4,17,32,316.3,101,174.05,4,4,13.0
73073,9981,2017/4/2,0,9,13,48,0,0,1.0,1.0,...,0,8,12,40,316.3,101,174.05,4,4,4.0
73074,9981,2017/4/16,0,5,14,49,0,0,1.5,1.4,...,0,4,12,42,316.3,101,174.05,4,4,9.0


In [88]:
date = set(data['order_time'])
print(date)

{'2016/7/10', '2016/10/16', '2017/3/19', '2016/8/7', '2016/5/1', '2016/9/18', '2016/9/4', '2017/4/2', '2017/4/30', '2016/7/24', '2016/6/26', '2017/1/8', '2017/2/19', '2016/11/13', '2016/10/30', '2016/6/12', '2017/1/22', '2016/12/11', '2017/2/5', '2016/10/2', '2016/12/25', '2016/11/27', '2017/4/16', '2016/5/29', '2016/8/21', '2016/5/15', '2017/3/5'}


In [89]:
set(data['cate'])

{1, 30, 46, 71, 83, 101}

In [31]:
# extract numeric features from date
order_time = data['order_time']
order_time_df = pd.to_datetime(order_time) 
data['year_order_time'] = order_time_df.dt.year 
data['month_order_time'] = order_time_df.dt.month 
data['day_order_time'] = order_time_df.dt.day 
data['hour_order_time'] = order_time_df.dt.hour
data['minute_order_time'] = order_time_df.dt.minute
data['second_order_time'] = order_time_df.dt.second
data['weekday_order_time'] = order_time_df.dt.weekday

In [32]:
# Apply One-Hot-Encoding to categorical features
def One_hot_encoding(dataframe, features):
    feature_dummy = []
    for i in features:
        dataframe_dummy = pd.get_dummies(dataframe[i], prefix = i)
        feature_dummy.append(dataframe_dummy)
    dataframe_new = pd.concat(feature_dummy + [dataframe], axis = 1)
    dataframe_new.drop(features, inplace = True, axis = 1)
    return dataframe_new

In [98]:
data.columns

Index(['sku_id', 'order_time', 'sales', 'sales_sum/2w', 'sales_sum/month',
       'sales_sum/quarter', 'comment_sum', 'comment_mean', 'comment_mean/2w',
       'comment_mean/month', 'comment_mean/quarter', 'comment_sum/2w',
       'comment_sum/month', 'comment_sum/quarter', 'browse', 'follow',
       'browse/2w', 'browse/month', 'browse/quarter', 'follow/2w',
       'follow/month', 'follow/quarter', 'user_lv', 'user_num', 'user_lv/2w',
       'user_lv/month', 'user_lv/quarter', 'user_num/2w', 'user_num/month',
       'user_num/quarter', 'area', 'area_sum/2w', 'area_sum/month',
       'area_sum/quarter', 'price', 'cate', 'para_1', 'para_2', 'para_3',
       'output', 'year_order_time', 'month_order_time', 'day_order_time',
       'hour_order_time', 'minute_order_time', 'second_order_time',
       'weekday_order_time'],
      dtype='object')

In [33]:
# apply one-hot encoding
feature_cols = ['cate', 'year_order_time', 'month_order_time']
data_oh = One_hot_encoding(data, feature_cols)

In [34]:
training_set = data_oh[data_oh['order_time'] != '2017/4/30']
training_set = training_set[training_set['order_time'] != '2017/4/16']
training_set = training_set[training_set['order_time'] != '2016/5/1']
training_set = training_set[training_set['order_time'] != '2016/10/30']
testing_set = data_oh[data_oh['order_time'] == '2017/4/16']
training_set.reset_index(inplace = True)
testing_set.reset_index(inplace = True)
del training_set['index']
del testing_set['index']

In [51]:
set(training_set['order_time'])

{'2016/10/16',
 '2016/10/2',
 '2016/10/30',
 '2016/11/13',
 '2016/12/11',
 '2016/12/25',
 '2016/5/15',
 '2016/5/29',
 '2016/6/12',
 '2016/6/26',
 '2016/7/10',
 '2016/7/24',
 '2016/8/21',
 '2016/8/7',
 '2016/9/18',
 '2016/9/4',
 '2017/1/22',
 '2017/1/8',
 '2017/2/19',
 '2017/2/5',
 '2017/3/19',
 '2017/3/5',
 '2017/4/2'}

In [52]:
set(testing_set['order_time'])

{'2017/4/16'}

In [35]:
training_set.index = training_set['sku_id']
testing_set.index = testing_set['sku_id']
drop_columns = ['sku_id', 'order_time']
training_set.drop(drop_columns, inplace = True, axis = 1)
testing_set.drop(drop_columns, inplace = True, axis = 1)

In [36]:
training_set.to_csv('/Users/mac/Desktop/UCB/290-Data-X/Project/Dataset/train.csv')
testing_set.to_csv('/Users/mac/Desktop/UCB/290-Data-X/Project/Dataset/test.csv')

In [50]:
training_set = pd.read_csv('/Users/mac/Desktop/UCB/290-Data-X/Project/Dataset/train.csv', index_col = 'sku_id')  
testing_set = pd.read_csv('/Users/mac/Desktop/UCB/290-Data-X/Project/Dataset/test.csv', index_col = 'sku_id')

In [51]:
sku_id = list(set(training_set.index))

In [70]:
sku_id

[16385,
 98308,
 32774,
 49159,
 90120,
 49162,
 49163,
 24592,
 73744,
 81939,
 32790,
 73753,
 16410,
 24603,
 24605,
 73759,
 49188,
 57385,
 73769,
 43,
 81964,
 57391,
 73776,
 98359,
 49210,
 8253,
 98369,
 49218,
 8259,
 49223,
 98375,
 49226,
 76,
 57422,
 32848,
 90202,
 32860,
 82014,
 16479,
 98,
 32867,
 41061,
 82022,
 90217,
 32876,
 57452,
 73838,
 8302,
 73844,
 32885,
 73845,
 16503,
 24697,
 16513,
 132,
 32901,
 73861,
 82055,
 65672,
 8328,
 49302,
 16536,
 65690,
 8349,
 90269,
 73894,
 32936,
 98472,
 82091,
 65708,
 24753,
 90291,
 57527,
 32952,
 73913,
 98494,
 57537,
 41155,
 90307,
 57541,
 16582,
 16586,
 24778,
 32973,
 41165,
 98512,
 209,
 16596,
 57557,
 73943,
 32986,
 219,
 57562,
 16607,
 65760,
 90335,
 98527,
 32995,
 82148,
 41189,
 41190,
 57574,
 8419,
 41193,
 49390,
 239,
 41200,
 41201,
 49393,
 49399,
 16633,
 24825,
 49403,
 65788,
 254,
 49409,
 260,
 8459,
 57617,
 24851,
 74010,
 82208,
 33059,
 74020,
 16677,
 57638,
 65831,
 33064,
 822

In [52]:
y_train = training_set['output']
training_set.drop(['output'], inplace = True, axis = 1)
X_train = training_set[:]
y_test = testing_set['output']
testing_set.drop(['output'], inplace = True, axis = 1)
X_test = testing_set[:]

In [41]:
y_train.values

array([ 0.,  0.,  0., ..., 12., 13.,  4.])

In [53]:
# Normalization processing to speed-up calculation
ss_X = StandardScaler()  
ss_y = StandardScaler()  

X_train_scaled = ss_X.fit_transform(X_train)  
X_test_scaled = ss_X.transform(X_test)

In [123]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / (y_true + 0.0001)))

# XGBoost

In [114]:
# Time series K-fold
tscv = TimeSeriesSplit(n_splits = 5)
mse_xgb_array = []
mae_xgb_array = []
mape_xgb_array = []
r2_xgb_array = []
startTime = time.time()
fold = 0

params = {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 4000, 
          'min_child_weight': 1, 'colsample_bytree': 0.6, 
          'subsample': 0.8, 'seed': 10}

# Model training
for train_index, val_index in tscv.split(X_train_scaled):
    fold += 1
    print('Training on fold %d, please wait...' % fold)
    X_train_cv, X_val_cv, y_train_cv, y_val_cv = (X_train_scaled[train_index], X_train_scaled[val_index], 
                                                  y_train[train_index], y_train[val_index])
    
    dtrain = xgb.DMatrix(X_train_cv, label = y_train_cv)
    dval = xgb.DMatrix(X_val_cv)
    model = xgb.train(params, dtrain, num_boost_round = 1000)
    prediction = model.predict(dval)
    
    mse = mean_squared_error(prediction, y_val_cv)
    mae = mean_absolute_error(prediction, y_val_cv)
    mape = mean_absolute_percentage_error(prediction, y_val_cv)
    r2 = r2_score(prediction, y_val_cv)
    
    mse_xgb_array.append(mse)
    mae_xgb_array.append(mae)
    mape_xgb_array.append(mape)
    r2_xgb_array.append(r2)
    
    print('Training on fold %d completed' % fold)

time_xgb = time.time() - startTime
print('The whole XGBoost takes %fs!' % time_xgb, end = '\n')

mse_xgb = np.mean(mse_xgb_array)
print('MSE Score:')  
print(mse_xgb, end = '\n')

mae_xgb = np.mean(mae_xgb_array)
print('MAE Score:')  
print(mae_xgb, end = '\n')

mape_xgb = np.mean(mape_xgb_array)
print('MAPE Score:')  
print(mape_xgb, end = '\n')

r2_xgb = np.mean(r2_xgb_array)
print('R^2 Score:')  
print(r2_xgb, end = '\n')

Training on fold 1, please wait...
Training on fold 1 completed
Training on fold 2, please wait...
Training on fold 2 completed
Training on fold 3, please wait...
Training on fold 3 completed
Training on fold 4, please wait...
Training on fold 4 completed
Training on fold 5, please wait...
Training on fold 5 completed
The whole XGBoost takes 258.668119s!
MSE Score:
1367.7731198997715
MAE Score:
7.483407580563354
MAPE Score:
0.8860681060018148
R^2 Score:
0.4964251718515634


In [125]:
params = {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 4000, 
          'min_child_weight': 1, 'colsample_bytree': 0.6, 
          'subsample': 0.8, 'seed': 10}

startTime = time.time()

# Model training
print('Training XGBoost, please wait...')
dtrain = xgb.DMatrix(X_train_scaled, label = y_train)
dval = xgb.DMatrix(X_test_scaled)
model = xgb.train(params, dtrain, num_boost_round = 1000)
prediction_xgb = model.predict(dval)

prediction_xgb = prediction_xgb * 0.8
print('Training on XGBoost completed')

mse_xgb = mean_squared_error(prediction_xgb, y_test)
mae_xgb = mean_absolute_error(prediction_xgb, y_test)
mape_xgb = mean_absolute_percentage_error(prediction_xgb, y_test)
r2_xgb = r2_score(prediction_xgb, y_test)

time_xgb = time.time() - startTime
print('The whole Quantile XGBoost takes %fs!' % time_xgb)
print()

print('MSE Score:')  
print(mse_xgb, end = '\n')

print('MAE Score:')  
print(mae_xgb, end = '\n')

print('MAPE Score:')  
print(mape_xgb, end = '\n')

print('R^2 Score:')  
print(r2_xgb, end = '\n')

Training XGBoost, please wait...
Training on XGBoost completed
The whole Quantile XGBoost takes 131.052861s!

MSE Score:
2136.314731211648
MAE Score:
11.350661437125106
MAPE Score:
0.7162362808166592
R^2 Score:
0.7705595921932032


In [143]:
testing_set = data[data['order_time'] == '2017/4/16']
testing_set.index = testing_set['sku_id']
testing_set_final = pd.concat((testing_set, y_test), axis = 1)
testing_set_final['prediction_xgb'] = prediction_xgb
testing_set_final['prediction_rf'] = prediction_rf
testing_set_final['prediction_lgb'] = prediction_lgb
testing_set_final['prediction_arima'] = prediction_arima
testing_set_final.to_csv('/Users/mac/Desktop/UCB/290-Data-X/Project/Results/result.csv', index = False)

In [146]:
prediction_all = prediction_rf * 0.5 + prediction_xgb * 0.4 + prediction_arima * 0.1
mse_xgb = mean_squared_error(prediction_all, y_test)
mae_xgb = mean_absolute_error(prediction_all, y_test)
mape_xgb = mean_absolute_percentage_error(prediction_all, y_test)
r2_xgb = r2_score(prediction_all, y_test)

time_xgb = time.time() - startTime
print('The whole Quantile XGBoost takes %fs!' % time_xgb)
print()

print('MSE Score:')  
print(mse_xgb, end = '\n')

print('MAE Score:')  
print(mae_xgb, end = '\n')

print('MAPE Score:')  
print(mape_xgb, end = '\n')

print('R^2 Score:')  
print(r2_xgb, end = '\n')

The whole Quantile XGBoost takes 327.145439s!

MSE Score:
2141.9181131067844
MAE Score:
11.668476366679696
MAPE Score:
0.7411809333502455
R^2 Score:
0.785911063108908


# Random Forest

In [124]:
# Time series K-fold
tscv = TimeSeriesSplit(n_splits = 5)
mse_rf_array = []
mae_rf_array = []
mape_rf_array = []
r2_rf_array = []
startTime = time.time()
fold = 0

# Model training
for train_index, val_index in tscv.split(X_train_scaled):
    fold += 1
    print('Training on fold %d, please wait...' % fold)
    X_train_cv, X_val_cv, y_train_cv, y_val_cv = (X_train_scaled[train_index], X_train_scaled[val_index], 
                                                  y_train[train_index], y_train[val_index])
    
    rf = RandomForestRegressor(n_estimators = 600, max_depth = 13, min_samples_split = 3,
                               min_samples_leaf = 10, oob_score = True, max_features = 'sqrt', 
                               random_state = 10)

    rf.fit(X_train_cv, y_train_cv)
    prediction = rf.predict(X_val_cv)
    
    mse = mean_squared_error(prediction, y_val_cv)
    mae = mean_absolute_error(prediction, y_val_cv)
    mape = mean_absolute_percentage_error(prediction, y_val_cv)
    r2 = r2_score(prediction, y_val_cv)
    
    mse_rf_array.append(mse)
    mae_rf_array.append(mae)
    mape_rf_array.append(mape)
    r2_rf_array.append(r2)
    
    print('Training on fold %d completed' % fold)

time_rf = time.time() - startTime
print('The whole Random Forest takes %fs!' % time_rf, end = '\n')

mse_rf = np.mean(mse_rf_array)
print('MSE Score:')  
print(mse_rf, end = '\n')

mae_rf = np.mean(mae_rf_array)
print('MAE Score:')  
print(mae_rf, end = '\n')

mape_rf = np.mean(mape_rf_array)
print('MAPE Score:')  
print(mape_rf, end = '\n')

r2_rf = np.mean(r2_rf_array)
print('R^2 Score:')  
print(r2_rf, end = '\n')


Training on fold 1, please wait...
Training on fold 1 completed
Training on fold 2, please wait...
Training on fold 2 completed
Training on fold 3, please wait...
Training on fold 3 completed
Training on fold 4, please wait...
Training on fold 4 completed
Training on fold 5, please wait...
Training on fold 5 completed
The whole Random Forest takes 222.537541s!
MSE Score:
1504.6800032107658
MAE Score:
7.444105386646619
MAPE Score:
0.9911027802408661
R^2 Score:
0.40914658573569157


In [129]:
startTime = time.time()

# Model training
print('Training Random Forest, please wait...')
rf = RandomForestRegressor(n_estimators = 1000, max_depth = 13, min_samples_split = 3,
                           min_samples_leaf = 10, oob_score = True, max_features = 'sqrt', 
                           random_state = 10)

rf.fit(X_train_scaled, y_train)
prediction_rf = rf.predict(X_test_scaled)

print('Training on Random Forest completed')

mse_rf = mean_squared_error(prediction_rf, y_test)
mae_rf = mean_absolute_error(prediction_rf, y_test)
mape_rf = mean_absolute_percentage_error(prediction_rf, y_test)
r2_rf = r2_score(prediction_rf, y_test)

time_rf = time.time() - startTime
print('The whole Random Forest takes %fs!' % time_rf)
print()

print('MSE Score:')  
print(mse_rf, end = '\n')

print('MAE Score:')  
print(mae_rf, end = '\n')

print('MAPE Score:')  
print(mape_rf, end = '\n')

print('R^2 Score:')  
print(r2_rf, end = '\n')

Training Random Forest, please wait...
Training on Random Forest completed
The whole Random Forest takes 153.967965s!

MSE Score:
2066.323610053949
MAE Score:
12.249568982063353
MAPE Score:
0.7611420514018358
R^2 Score:
0.7917750627800018


# LightGBM

In [130]:
# Time series K-fold
tscv = TimeSeriesSplit(n_splits = 5)
mse_lgb_array = []
mae_lgb_array = []
mape_lgb_array = []
r2_lgb_array = []
startTime = time.time()
fold = 0

# Model training
for train_index, val_index in tscv.split(X_train_scaled):
    fold += 1
    print('Training on fold %d, please wait...' % fold)
    X_train_cv, X_val_cv, y_train_cv, y_val_cv = (X_train_scaled[train_index], X_train_scaled[val_index], 
                                                  y_train[train_index], y_train[val_index])
    
    lgb_params = {'n_estimators': 4000, 'max_depth': 8, 'min_data_in_leaf': 10, 
                  'subsample': 0.9, 'learning_rate': 0.1, 'colsample_bytree': 0.9, 
                  'boosting_type': 'gbdt', 'random_state': 10}
    model = lgb.LGBMRegressor(objective = 'regression', **lgb_params)
    model.fit(X_train_cv, y_train_cv)
    prediction = model.predict(X_val_cv)

    mse = mean_squared_error(prediction, y_val_cv)
    mae = mean_absolute_error(prediction, y_val_cv)
    mape = mean_absolute_percentage_error(prediction, y_val_cv)
    r2 = r2_score(prediction, y_val_cv)
    
    mse_lgb_array.append(mse)
    mae_lgb_array.append(mae)
    mape_lgb_array.append(mape)
    r2_lgb_array.append(r2)
    
    print('Training on fold %d completed' % fold)

time_lgb = time.time() - startTime
print('The whole LightGBM takes %fs!' % time_lgb, end = '\n')

mse_lgb = np.mean(mse_lgb_array)
print('MSE Score:')  
print(mse_lgb, end = '\n')

mae_lgb = np.mean(mae_lgb_array)
print('MAE Score:')  
print(mae_lgb, end = '\n')

mape_lgb = np.mean(mape_lgb_array)
print('MAPE Score:')  
print(mape_lgb, end = '\n')

r2_lgb = np.mean(r2_lgb_array)
print('R^2 Score:')  
print(r2_lgb, end = '\n')

Training on fold 1, please wait...
Training on fold 1 completed
Training on fold 2, please wait...
Training on fold 2 completed
Training on fold 3, please wait...
Training on fold 3 completed
Training on fold 4, please wait...
Training on fold 4 completed
Training on fold 5, please wait...
Training on fold 5 completed
The whole LightGBM takes 217.695097s!
MSE Score:
1746.689439498652
MAE Score:
7.924171892545134
MAPE Score:
3.2718696167649517
R^2 Score:
0.4510815537830883


In [130]:
startTime = time.time()

# Model training
print('Training LightGBM, please wait...')
lgb_params = {'n_estimators': 4000, 'max_depth': 8, 'min_data_in_leaf': 10, 
              'subsample': 0.9, 'learning_rate': 0.1, 'colsample_bytree': 0.9, 
              'boosting_type': 'gbdt', 'random_state': 10}
model = lgb.LGBMRegressor(objective = 'regression', **lgb_params)
model.fit(X_train_scaled, y_train)
prediction_lgb = model.predict(X_test_scaled)

print('Training on LightGBM completed')

mse_lgb = mean_squared_error(prediction_lgb, y_test)
mae_lgb = mean_absolute_error(prediction_lgb, y_test)
mape_lgb = mean_absolute_percentage_error(prediction_lgb, y_test)
r2_lgb = r2_score(prediction_lgb, y_test)

time_lgb = time.time() - startTime
print('The whole LightGBM takes %fs!' % time_lgb)
print()

print('MSE Score:')  
print(mse_lgb, end = '\n')

print('MAE Score:')  
print(mae_lgb, end = '\n')

print('MAPE Score:')  
print(mape_lgb, end = '\n')

print('R^2 Score:')  
print(r2_lgb, end = '\n')

Training LightGBM, please wait...
Training on LightGBM completed
The whole LightGBM takes 45.779983s!

MSE Score:
3315.496774028581
MAE Score:
16.798301303247364
MAPE Score:
21.032191139890923
R^2 Score:
0.7447772452999813


# Arima

In [72]:
from pandas.plotting import autocorrelation_plot
from matplotlib import pyplot

%matplotlib
y_train_tmp = list(y_train.loc[16385])
autocorrelation_plot(y_train_tmp)
pyplot.show()


Using matplotlib backend: MacOSX


In [141]:
startTime = time.time()

# Model training
print('Training ARIMA, please wait...')

prediction_arima = []
y_train_tmp = list(y_train)

prediction_df = pd.DataFrame([], index = y_test.index, columns = ['prediction'])

for sku in sku_id:
    y_train_tmp = list(y_train.loc[sku])
    #print(y_train_tmp)
    #print(len(y_train_tmp))
    model = SARIMAX(y_train_tmp)
    model_fit = model.fit()
    prediction = model_fit.predict(start = len(y_train_tmp), end = len(y_train_tmp))
    prediction_df['prediction'].loc[sku] = prediction[0]
    
prediction_arima = np.reshape(prediction_df.values, y_test.values.shape, )
print('Training on ARIMA completed')

mse_arima = mean_squared_error(prediction_arima, y_test)
mae_arima = mean_absolute_error(prediction_arima, y_test)
mape_arima = mean_absolute_percentage_error(prediction_arima, y_test.values)
r2_arima = r2_score(prediction_arima, y_test)

time_arima = time.time() - startTime
print('The whole ARIMA takes %fs!' % time_arima)
print()

print('MSE Score:')  
print(mse_arima, end = '\n')

print('MAE Score:')  
print(mae_arima, end = '\n')

print('MAPE Score:')  
print(mape_arima, end = '\n')

print('R^2 Score:')  
print(r2_arima, end = '\n')

Training ARIMA, please wait...
Training on ARIMA completed
The whole ARIMA takes 86.671394s!

MSE Score:
4392.380654143197
MAE Score:
13.457189962564083
MAPE Score:
3098.2397103551652
R^2 Score:
0.7076162344679846
